In [15]:
import os
import argparse
import numpy as np
import tensorflow as tf
from collections import namedtuple
import shutil
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style

In [16]:
def model_path():
    idx = 0
    path = os.path.join('models', 'model-{}')
    while os.path.exists(path.format(idx)):
        idx += 1
    path = path.format(idx)
    os.makedirs(os.path.join(path, 'models'))
    os.makedirs(os.path.join(path, 'backup'))
    for file in filter(lambda x: x.endswith('.py'), os.listdir('.')):
        shutil.copy2(file, os.path.join(path, 'backup'))
    return path

## 1) Set the hyperparameters

In [23]:
seq_len = 256
batch_size = 64
num_epoch = 30
window_mixtures = 10
output_mixtures = 20
lstm_layers = 3
units_per_layer = 400
# change this to the path where the model was saved if you wish to continue trainng a half trained model
restore_model = None 
batches_per_epoch = 50
epsilon = 1e-8

## 2) Define the window layer<br/>

**a) Calculate the parameters**
<br/><br/>
$(\alpha', \beta', \kappa') = W_{h^1p}h_t + b_p$
<br/><br/>
$\alpha = exp(\alpha')$
<br/><br/>
$\beta = exp(\beta')$
<br/><br/>
$\kappa = k_{t-1} + exp(\kappa')$
<br/><br/>

**b) Calculate phi**
<br/><br/>
$\phi(t, u) = \sum_{k=1}^{K} {\alpha_t^k exp(-\beta^k(\kappa^k - u)^2)}$
<br/><br/>
**c) Now, finally, calculate the window weights**
<br/><br/>
$w_t = \sum_{u=1}^{U}{\phi(t, u)c_u}$

In [24]:
class WindowLayer(object):
    def __init__(self, num_mixtures, sequence, num_letters):
        self.sequence = sequence  # one-hot encoded sequence of characters -- [batch_size, length, num_letters]
        self.seq_len = tf.shape(sequence)[1]
        self.num_mixtures = num_mixtures
        self.num_letters = num_letters
        self.u_range = -tf.expand_dims(tf.expand_dims(tf.range(0., tf.cast(self.seq_len, dtype=tf.float32)), axis=0),
                                       axis=0)

    def __call__(self, inputs, k, reuse=None):
        with tf.variable_scope('window', reuse=reuse):
            alpha = tf.layers.dense(inputs, self.num_mixtures, activation=tf.exp,
                                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='alpha')
            beta = tf.layers.dense(inputs, self.num_mixtures, activation=tf.exp,
                                   kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='beta')
            kappa = tf.layers.dense(inputs, self.num_mixtures, activation=tf.exp,
                                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='kappa')

            a = tf.expand_dims(alpha, axis=2)
            b = tf.expand_dims(beta, axis=2)
            k = tf.expand_dims(k + kappa, axis=2)

            phi = tf.exp(-np.square(self.u_range + k) * b) * a  # [batch_size, mixtures, length]
            phi = tf.reduce_sum(phi, axis=1, keepdims=True)  # [batch_size, 1, length]

            # whether or not network finished generating sequence
            finish = tf.cast(phi[:, 0, -1] > tf.reduce_max(phi[:, 0, :-1], axis=1), tf.float32)

            return tf.squeeze(tf.matmul(phi, self.sequence), axis=1), \
                   tf.squeeze(k, axis=2), \
                   tf.squeeze(phi, axis=1), \
                   tf.expand_dims(finish, axis=1)

    @property
    def output_size(self):
        return [self.num_letters, self.num_mixtures, 1]

## 3) Define the Mixture layer<br/>

**Calculate the parameters**

$y_t' = (e_t',(\pi_t'^j, \mu_t'^j, \sigma_t'^j, \rho_t'^j)_{m=1}^{M}) = b_y + \sum_{n=1}^{N}{W_{h^ny}h_t^n}$
<br/><br/>
$e_t = \frac{1}{1 + exp(e_t')}\,\,\,\,\,(sigmoid)$
<br/><br/>
$\pi_t^j = \frac{exp(\pi_t'^j)}{\sum_{j'=1}^{M}{exp(\pi_t'^j)}}\,\,\,\,\,(softmax)$
<br/><br/>
$\mu_t^j = \mu_t'^j$
<br/><br/>
$\sigma_t^j = exp(\sigma_t'^j)$
<br/><br/>
$\rho_t^j = \tanh(\rho_t'^j)$
<br/><br/>

In [25]:
class MixtureLayer(object):
    def __init__(self, input_size, output_size, num_mixtures):
        self.input_size = input_size
        self.output_size = output_size
        self.num_mixtures = num_mixtures

    def __call__(self, inputs, bias=0., reuse=None):
        with tf.variable_scope('mixture_output', reuse=reuse):
            e = tf.layers.dense(inputs, 1,
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='e')
            pi = tf.layers.dense(inputs, self.num_mixtures,
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='pi')
            mu1 = tf.layers.dense(inputs, self.num_mixtures,
                                  kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='mu1')
            mu2 = tf.layers.dense(inputs, self.num_mixtures,
                                  kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='mu2')
            std1 = tf.layers.dense(inputs, self.num_mixtures,
                                   kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='std1')
            std2 = tf.layers.dense(inputs, self.num_mixtures,
                                   kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='std2')
            rho = tf.layers.dense(inputs, self.num_mixtures,
                                  kernel_initializer=tf.truncated_normal_initializer(stddev=0.075), name='rho')

            return tf.nn.sigmoid(e), \
                   tf.nn.softmax(pi * (1. + bias)), \
                   mu1, mu2, \
                   tf.exp(std1 - bias), tf.exp(std2 - bias), \
                   tf.nn.tanh(rho)

## 4) Define the RNN model

In [26]:
class RNNModel(tf.nn.rnn_cell.RNNCell):
    def __init__(self, layers, num_units, input_size, num_letters, batch_size, window_layer):
        super(RNNModel, self).__init__()
        self.layers = layers
        self.num_units = num_units
        self.input_size = input_size
        self.num_letters = num_letters
        self.window_layer = window_layer
        self.last_phi = None

        with tf.variable_scope('rnn', reuse=None):
            self.lstms = [tf.nn.rnn_cell.LSTMCell(num_units)
                          for _ in range(layers)]
            self.states = [tf.Variable(tf.zeros([batch_size, s]), trainable=False)
                           for s in self.state_size]

            self.zero_states = tf.group(*[sp.assign(sc)
                                          for sp, sc in zip(self.states,
                                                            self.zero_state(batch_size, dtype=tf.float32))])

    @property
    def state_size(self):
        return [self.num_units] * self.layers * 2 + self.window_layer.output_size

    @property
    def output_size(self):
        return [self.num_units]

    def call(self, inputs, state, **kwargs):
        # state[-3] --> window
        # state[-2] --> k
        # state[-1] --> finish
        # state[2n] --> h
        # state[2n+1] --> c
        window, k, finish = state[-3:]
        output_state = []
        prev_output = []

        for layer in range(self.layers):
            x = tf.concat([inputs, window] + prev_output, axis=1)
            with tf.variable_scope('lstm_{}'.format(layer)):
                output, s = self.lstms[layer](x, tf.nn.rnn_cell.LSTMStateTuple(state[2 * layer],
                                                                               state[2 * layer + 1]))
                prev_output = [output]
            output_state += [*s]

            if layer == 0:
                window, k, self.last_phi, finish = self.window_layer(output, k)

        return output, output_state + [window, k, finish]

## 5) Loss function
* Calculate the probability of next input being $x_{t+1}$ for output vector $y_t$ is
<br/><br/>
$P(x_{t+1}|y_t) = \sum_{j=1}^{M}\pi_t^jN(x_{t+1}|\mu_t^j, \sigma_t^j, \rho_t^j) 
\begin{cases}
  e_t & \text{if }(x_{t+1})_3 = 1 \\    
  1-e_t & \text{otherwise}\\  
\end{cases}$
<br/><br/>
where
<br/><br/>
$N(x, \mu, \sigma, \rho) = \frac{1}{2\pi\sigma_1\sigma_2\sqrt{1 - \rho^2}}exp\,(\frac{-Z}{2(1 - \rho^2)})$
<br/><br/>
$Z = \frac{(x_1 - \mu_1)^2}{\sigma_1^2} + \frac{(x_2 - \mu_2)^2}{\sigma_2^2} 
+ \frac{2\rho(x_1 - \mu_1)(x_2 - \mu_2)}{\sigma_1\sigma_2}$
<br/><br/>
* The loss function is given by taking it's negative logarithm and adding them up over all timesteps
<br/><br/>
$L(x) = \sum_{t=1}^{T}-\log({\sum_{j=1}^{M}\pi_t^jN(x_{t+1}|\mu_t^j, \sigma_t^j, \rho_t^j)}) - 
\begin{cases}
    log(e_t) & \text{if }(x_{t+1})_3 = 1 \\    
    log(1-e_t) & \text{otherwise}\\  
\end{cases}$

In [27]:
def create_graph(num_letters, batch_size,
                 num_units=400, lstm_layers=3,
                 window_mixtures=10, output_mixtures=20):
    graph = tf.Graph()
    with graph.as_default():
        coordinates = tf.placeholder(tf.float32, shape=[None, None, 3])
        sequence = tf.placeholder(tf.float32, shape=[None, None, num_letters])
        reset = tf.placeholder(tf.float32, shape=[None, 1])
        bias = tf.placeholder_with_default(tf.zeros(shape=[]), shape=[])

        def create_model(generate=None):
            in_coords = coordinates[:, :-1, :]
            out_coords = coordinates[:, 1:, :]

            _batch_size = 1 if generate else batch_size
            if generate:
                in_coords = coordinates

            with tf.variable_scope('model', reuse=generate):
                window = WindowLayer(num_mixtures=window_mixtures, sequence=sequence, num_letters=num_letters)

                rnn_model = RNNModel(layers=lstm_layers, num_units=num_units,
                                     input_size=3, num_letters=num_letters,
                                     window_layer=window, batch_size=_batch_size)

                reset_states = tf.group(*[state.assign(state * reset)
                                          for state in rnn_model.states])

                outs, states = tf.nn.dynamic_rnn(rnn_model, in_coords,
                                                 initial_state=rnn_model.states)

                output_layer = MixtureLayer(input_size=num_units, output_size=2,
                                            num_mixtures=output_mixtures)

                with tf.control_dependencies([sp.assign(sc) for sp, sc in zip(rnn_model.states, states)]):
                    with tf.name_scope('prediction'):
                        outs = tf.reshape(outs, [-1, num_units])
                        e, pi, mu1, mu2, std1, std2, rho = output_layer(outs, bias)

                    with tf.name_scope('loss'):
                        coords = tf.reshape(out_coords, [-1, 3])
                        xs, ys, es = tf.unstack(tf.expand_dims(coords, axis=2), axis=1)

                        mrho = 1 - tf.square(rho)
                        xms = (xs - mu1) / std1
                        yms = (ys - mu2) / std2
                        z = tf.square(xms) + tf.square(yms) - 2. * rho * xms * yms
                        n = 1. / (2. * np.pi * std1 * std2 * tf.sqrt(mrho)) * tf.exp(-z / (2. * mrho))
                        ep = es * e + (1. - es) * (1. - e)
                        rp = tf.reduce_sum(pi * n, axis=1)

                        loss = tf.reduce_mean(-tf.log(rp + epsilon) - tf.log(ep + epsilon))

                    if generate:
                        # save params for easier model loading and prediction
                        for param in [('coordinates', coordinates),
                                      ('sequence', sequence),
                                      ('bias', bias),
                                      ('e', e), ('pi', pi),
                                      ('mu1', mu1), ('mu2', mu2),
                                      ('std1', std1), ('std2', std2),
                                      ('rho', rho),
                                      ('phi', rnn_model.last_phi),
                                      ('window', rnn_model.states[-3]),
                                      ('kappa', rnn_model.states[-2]),
                                      ('finish', rnn_model.states[-1]),
                                      ('zero_states', rnn_model.zero_states)]:
                            tf.add_to_collection(*param)

                with tf.name_scope('training'):
                    steps = tf.Variable(0.)
                    learning_rate = tf.train.exponential_decay(0.001, steps, staircase=True,
                                                               decay_steps=10000, decay_rate=0.5)

                    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
                    grad, var = zip(*optimizer.compute_gradients(loss))
                    grad, _ = tf.clip_by_global_norm(grad, 3.)
                    train_step = optimizer.apply_gradients(zip(grad, var), global_step=steps)

                with tf.name_scope('summary'):
                    # TODO: add more summaries
                    summary = tf.summary.merge([
                        tf.summary.scalar('loss', loss)
                    ])

                return namedtuple('Model', ['coordinates', 'sequence', 'reset_states', 'reset', 'loss', 'train_step',
                                            'learning_rate', 'summary'])(
                           coordinates, sequence, reset_states, reset, loss, train_step, learning_rate, summary
                       )
        train_model = create_model(generate=None)
        _ = create_model(generate=True)  # just to create ops for generation

    return graph, train_model

## 6) All set, let's train the model

In [54]:
# realtime plotting of graph
plot_realtime = False
if plot_realtime:
    style.use('fivethirtyeight')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.ion()

    plt.show()
    fig.canvas.draw()

def live_plot(loss_vals):
    ax.clear()
    ax.plot(loss_vals)
    fig.canvas.draw()

losses = []
# train the model
%run data_generator.ipynb
batch_generator = DataGenerator(batch_size, seq_len)
g, vs = create_graph(batch_generator.num_chars, batch_size,
                     num_units=units_per_layer, lstm_layers=lstm_layers,
                     window_mixtures=window_mixtures,
                     output_mixtures=output_mixtures)
graph_count = 0
with tf.Session(graph=g) as sess:
    model_saver = tf.train.Saver(max_to_keep=2)
    if restore_model:
        model_file = tf.train.latest_checkpoint(os.path.join(restore_model, 'models'))
        mod_path = restore_model
        epoch = int(model_file.split('-')[-1]) + 1
        model_saver.restore(sess, model_file)
    else:
        sess.run(tf.global_variables_initializer())
        mod_path = model_path()
        epoch = 0

    summary_writer = tf.summary.FileWriter(mod_path, graph=g, flush_secs=10)
    summary_writer.add_session_log(tf.SessionLog(status=tf.SessionLog.START),
                                   global_step=epoch * batches_per_epoch)

    for e in range(epoch, num_epoch):
        print('\nEpoch {}'.format(e))
        for b in range(1, batches_per_epoch + 1):
            coords, seq, reset, needed = batch_generator.next_batch()
            if needed:
                sess.run(vs.reset_states, feed_dict={vs.reset: reset})
            l, s, _ = sess.run([vs.loss, vs.summary, vs.train_step],
                               feed_dict={vs.coordinates: coords,
                                          vs.sequence: seq})
            losses.append(l)
            graph_count += 1

            if plot_realtime and graph_count == 1:
                live_plot(losses)
                graph_count = 0

            summary_writer.add_summary(s, global_step=e * batches_per_epoch + b)
            #print('\r[{:5d}/{:5d}] loss = {}'.format(b, batches_per_epoch, l), end='')

        model_saver.save(sess, os.path.join(mod_path, 'models', 'model'),
                         global_step=e)

I trained the model on Google Colab for 5-6 hours and , the model spits out negative values of losses after a certain number of ephocs and seemed to worked well so no issue with that. The code also keeps on plotting a graph in realtime as per the number of epochs so that the changes loss can be supervised with training simultaneously, just change the "plot_realtime" variable to true and uncomment print statement to print the losses for each epoch